#### Setup

In [1]:
import os
import time
import sys
from dataclasses import dataclass
from tqdm import tqdm
import numpy as np
from numpy.random import Generator
import torch as t
from torch import Tensor
from torch.optim.optimizer import Optimizer
import gym
import gym.envs.registration
from gym.envs.classic_control.cartpole import CartPoleEnv
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical
import einops
from pathlib import Path
from typing import Tuple, Literal, Union, Optional
from jaxtyping import Float, Int
import wandb
from IPython.display import clear_output
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import warnings
warnings.filterwarnings('ignore')
# !pip install opencv-python
# Make sure exercises are in the path
chapter = "chapter2_rl"
exercises_dir = Path(f"{os.getcwd().split(chapter)[0]}/{chapter}/exercises").resolve()
section_dir = exercises_dir / "part3_ppo"
if str(exercises_dir) not in sys.path: sys.path.append(str(exercises_dir))

from part2_q_learning_and_dqn.utils import set_global_seeds
from part2_q_learning_and_dqn.solutions import Probe1, Probe2, Probe3, Probe4, Probe5
from part3_ppo.utils import make_env
import part3_ppo.utils as utils
import part3_ppo.tests as tests
from plotly_utils import plot_cartpole_obs_and_dones

# Register our probes from last time
for idx, probe in enumerate([Probe1, Probe2, Probe3, Probe4, Probe5]):
    gym.envs.registration.register(id=f"Probe{idx+1}-v0", entry_point=probe)

Arr = np.ndarray

device = t.device('mps' if t.backends.mps.is_available() else 'cuda' if t.cuda.is_available() else 'cpu')

MAIN = __name__ == "__main__"

/home/bart/.local/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn
/home/bart/.local/lib/python3.10/site-packages/wandb/util.py:152: SentryHubDeprecationWarning: `sentry_sdk.Hub` is deprecated and will be removed in a future major release. Please consult our 1.x to 2.x migration guide for details on how to migrate `Hub` usage to the new API: https://docs.sentry.io/platforms/python/migration/1.x-to-2.x
  sentry_hub = sentry_sdk.Hub(sentry_client)


#### Agent and Learning Phase

In [2]:
@dataclass
class PPOArgs:
    # Basic / global
    seed: int = 1
    cuda: bool = t.cuda.is_available()
    env_id: str = "CartPole-v1"
    mode: Literal["classic-control", "atari", "mujoco"] = "classic-control"

    # Wandb / logging
    use_wandb: bool = False
    capture_video: bool = True
    exp_name: str = "PPO_Implementation"
    log_dir: str = "logs"
    wandb_project_name: str = "PPOCart"
    wandb_entity: str = None

    # Duration of different phases
    total_timesteps: int = 500000
    num_envs: int = 4
    num_steps: int = 128
    num_minibatches: int = 4
    batches_per_learning_phase: int = 4

    # Optimization hyperparameters
    learning_rate: float = 2.5e-4
    max_grad_norm: float = 0.5

    # Computing advantage function
    gamma: float = 0.99
    gae_lambda: float = 0.95

    # Computing other loss functions
    clip_coef: float = 0.2
    ent_coef: float = 0.01
    vf_coef: float = 0.25

    def __post_init__(self):
        self.batch_size = self.num_steps * self.num_envs
        assert self.batch_size % self.num_minibatches == 0, "batch_size must be divisible by num_minibatches"
        self.minibatch_size = self.batch_size // self.num_minibatches
        self.total_phases = self.total_timesteps // self.batch_size
        self.total_training_steps = self.total_phases * self.batches_per_learning_phase * self.num_minibatches

args = PPOArgs(num_minibatches=2)
utils.arg_help(args)

,default value,description
arg,,
seed,1,seed of the experiment
cuda,False,"if toggled, cuda will be enabled by default"
env_id,'CartPole-v1',the id of the environment
mode,'classic-control',"can be 'classic-control', 'atari' or 'mujoco'"
use_wandb,False,"if toggled, this experiment will be tracked with Weights and Biases"
capture_video,True,whether to capture videos of the agent performances (check out `videos` folder)
exp_name,'PPO_Implementation',the name of this experiment
log_dir,'logs',the directory where the logs will be stored
wandb_project_name,'PPOCart',the wandb's project name


In [3]:
def layer_init(layer: nn.Linear, std=np.sqrt(2), bias_const=0.0):
    t.nn.init.orthogonal_(layer.weight, std)
    t.nn.init.constant_(layer.bias, bias_const)
    return layer


def get_actor_and_critic(
    envs: gym.vector.SyncVectorEnv,
    mode: Literal["classic-control", "atari", "mujoco"] = "classic-control",
) -> tuple[nn.Module, nn.Module]:
    '''
    Returns (actor, critic), the networks used for PPO, in one of 3 different modes.
    '''
    assert mode in ["classic-control", "atari", "mujoco"]

    obs_shape = envs.single_observation_space.shape
    num_obs = np.array(obs_shape).prod()
    num_actions = (
        envs.single_action_space.n
        if isinstance(envs.single_action_space, gym.spaces.Discrete)
        else np.array(envs.single_action_space.shape).prod()
    )

    if mode == "classic-control":
        actor, critic = get_actor_and_critic_classic(num_obs, num_actions)
    if mode == "atari":
        actor, critic = get_actor_and_critic_atari(obs_shape, num_actions)
    if mode == "mujoco":
        actor, critic = get_actor_and_critic_mujoco(num_obs, num_actions)

    return actor.to(device), critic.to(device)


def get_actor_and_critic_classic(num_obs: int, num_actions: int):
    '''
    Returns (actor, critic) in the "classic-control" case, according to diagram above.
    '''
    actor = nn.Sequential(
        layer_init(nn.Linear(num_obs, 64)),
        nn.Tanh(),
        layer_init(nn.Linear(64,64)),
        nn.Tanh(),
        layer_init(nn.Linear(64, num_actions), std=0.01)
    )
    critic = nn.Sequential(
        layer_init(nn.Linear(num_obs, 64)),
        nn.Tanh(),
        layer_init(nn.Linear(64,64)),
        nn.Tanh(),
        layer_init(nn.Linear(64, 1), std=1)
    )
    return actor, critic
    

tests.test_get_actor_and_critic(get_actor_and_critic, mode="classic-control")

All tests in `test_get_actor_and_critic(mode='classic-control')` passed!


In [4]:
@t.inference_mode()
def compute_advantages(
    next_value: t.Tensor,
    next_done: t.Tensor,
    rewards: t.Tensor,
    values: t.Tensor,
    dones: t.Tensor,
    gamma: float,
    gae_lambda: float,
) -> t.Tensor:
    '''Compute advantages using Generalized Advantage Estimation.
    next_value: shape (env,)
    next_done: shape (env,)
    rewards: shape (buffer_size, env)
    values: shape (buffer_size, env)
    dones: shape (buffer_size, env)
    Return: shape (buffer_size, env)
    '''
    T = values.shape[0]
    next_values = t.concat([values[1:], next_value.unsqueeze(0)])
    next_dones = t.concat([dones[1:], next_done.unsqueeze(0)])
    deltas = rewards + gamma * next_values * (1.0 - next_dones) - values
    advantages = t.zeros_like(deltas)
    advantages[-1] = deltas[-1]
    for s in reversed(range(1, T)):
        advantages[s-1] = deltas[s-1] + gamma * gae_lambda * (1.0 - dones[s]) * advantages[s]
    return advantages

tests.test_compute_advantages(compute_advantages)

All tests in `test_compute_advantages_single` passed!


In [5]:
def minibatch_indexes(rng: Generator, batch_size: int, minibatch_size: int) -> list[np.ndarray]:
    '''
    Return a list of length num_minibatches = (batch_size // minibatch_size), where each element is an
    array of indexes into the batch. Each index should appear exactly once.

    To relate this to the diagram above: if we flatten the non-shuffled experiences into:

        [1,1,1,1,2,2,2,2,3,3,3,3]

    then the output of this function could be the following list of arrays:

        [array([0,5,4,3]), array([11,6,7,8]), array([1,2,9,10])]

    which would give us the minibatches seen in the first row of the diagram above:

        [array([1,2,2,1]), array([3,2,2,3]), array([1,1,3,3])]
    '''
    assert batch_size % minibatch_size == 0
    # SOLUTION
    indices = rng.permutation(batch_size)
    indices = einops.rearrange(indices, "(mb_num mb_size) -> mb_num mb_size", mb_size=minibatch_size)
    return list(indices)

In [6]:
def to_numpy(arr: Union[np.ndarray, Tensor]):
    '''
    Converts a (possibly cuda and non-detached) tensor to numpy array.
    '''
    if isinstance(arr, Tensor):
        arr = arr.detach().cpu().numpy()
    return arr


@dataclass
class ReplayMinibatch:
    '''
    Samples from the replay memory, converted to PyTorch for use in neural network training.

    Data is equivalent to (s_t, a_t, logpi(a_t|s_t), A_t, A_t + V(s_t), d_t)
    '''    
    observations: Tensor # shape [minibatch_size, *observation_shape]
    actions: Tensor # shape [minibatch_size,, *action_shape]
    logprobs: Tensor # shape [minibatch_size,]
    advantages: Tensor # shape [minibatch_size,]
    returns: Tensor # shape [minibatch_size,]
    dones: Tensor # shape [minibatch_size,]


class ReplayMemory:
    '''
    Contains buffer; has a method to sample from it to return a ReplayMinibatch object.
    '''
    rng: Generator
    observations: np.ndarray # shape [buffer_size, num_envs, *observation_shape]
    actions: np.ndarray # shape [buffer_size, num_envs, *action_shape]
    logprobs: np.ndarray # shape [buffer_size, num_envs]
    values: np.ndarray # shape [buffer_size, num_envs]
    rewards: np.ndarray # shape [buffer_size, num_envs]
    dones: np.ndarray # shape [buffer_size, num_envs]

    def __init__(self, args: PPOArgs, envs: gym.vector.SyncVectorEnv):
        self.args = args
        self.rng = np.random.default_rng(args.seed)
        self.num_envs = envs.num_envs
        self.obs_shape = envs.single_observation_space.shape
        self.action_shape = envs.single_action_space.shape
        self.reset_memory()


    def reset_memory(self):
        '''
        Resets all stored experiences, ready for new ones to be added to memory.
        '''
        self.observations = np.empty((0, self.num_envs, *self.obs_shape), dtype=np.float32)
        self.actions = np.empty((0, self.num_envs, *self.action_shape), dtype=np.int32)
        self.logprobs = np.empty((0, self.num_envs), dtype=np.float32)
        self.values = np.empty((0, self.num_envs), dtype=np.float32)
        self.rewards = np.empty((0, self.num_envs), dtype=np.float32)
        self.dones = np.empty((0, self.num_envs), dtype=bool)


    def add(self, obs, actions, logprobs, values, rewards, dones) -> None:
        '''
        Each argument can be a PyTorch tensor or NumPy array.

        obs: shape (num_environments, *observation_shape)
            Observation before the action
        actions: shape (num_environments,)
            Action chosen by the agent
        logprobs: shape (num_environments,)
            Log probability of the action that was taken (according to old policy)
        values: shape (num_environments,)
            Values, estimated by the critic (according to old policy)
        rewards: shape (num_environments,)
            Reward after the action
        dones: shape (num_environments,)
            If True, the episode ended and was reset automatically
        '''
        assert obs.shape == (self.num_envs, *self.obs_shape)
        assert actions.shape == (self.num_envs, *self.action_shape)
        assert logprobs.shape == (self.num_envs,)
        assert values.shape == (self.num_envs,)
        assert dones.shape == (self.num_envs,)
        assert rewards.shape == (self.num_envs,)

        self.observations = np.concatenate((self.observations, to_numpy(obs[None, :])))
        self.actions = np.concatenate((self.actions, to_numpy(actions[None, :])))
        self.logprobs = np.concatenate((self.logprobs, to_numpy(logprobs[None, :])))
        self.values = np.concatenate((self.values, to_numpy(values[None, :])))
        self.rewards = np.concatenate((self.rewards, to_numpy(rewards[None, :])))
        self.dones = np.concatenate((self.dones, to_numpy(dones[None, :])))


    def get_minibatches(self, next_value: t.Tensor, next_done: t.Tensor):
        minibatches = []

        # Stack all experiences, and move them to our device
        obs, actions, logprobs, values, rewards, dones = [t.from_numpy(exp).to(device) for exp in [
            self.observations, self.actions, self.logprobs, self.values, self.rewards, self.dones
        ]]

        # Compute advantages and returns (then get the list of tensors, in the right order to add to our ReplayMinibatch)
        advantages = compute_advantages(next_value, next_done, rewards, values, dones.float(), self.args.gamma, self.args.gae_lambda)
        returns = advantages + values
        replay_memory_data = [obs, actions, logprobs, advantages, returns, dones]

        # Generate `batches_per_learning_phase` sets of minibatches (each set of minibatches is a shuffled permutation of
        # all the experiences stored in memory)
        for _ in range(self.args.batches_per_learning_phase):

            indices_for_each_minibatch = minibatch_indexes(self.rng, self.args.batch_size, self.args.minibatch_size)

            for indices_for_minibatch in indices_for_each_minibatch:
                minibatches.append(ReplayMinibatch(*[
                    arg.flatten(0, 1)[indices_for_minibatch] for arg in replay_memory_data
                ]))

        # Reset memory, since we only run this once per learning phase
        self.reset_memory()

        return minibatches

In [7]:
args = PPOArgs()
envs = gym.vector.SyncVectorEnv([make_env("CartPole-v1", i, i, False, "test") for i in range(4)])
next_value = t.zeros(envs.num_envs).to(device)
next_done = t.zeros(envs.num_envs).to(device)
memory = ReplayMemory(args, envs)
obs = envs.reset()

for i in range(args.num_steps):
    actions = envs.action_space.sample()
    (next_obs, rewards, dones, infos) = envs.step(actions)
    # just dummy values for now, we won't be using them
    logprobs = values = t.zeros(envs.num_envs)
    # add everything to buffer (the casting from arrays to tensors is handled for us)
    memory.add(obs, actions, logprobs, values, rewards, dones)
    obs = next_obs

obs = memory.observations # shape [num_steps, num_envs, obs_shape=4]
dones = memory.dones # shape [num_steps, num_envs]

plot_cartpole_obs_and_dones(obs, dones, title="CartPole experiences (dotted lines = termination, solid lines = environment separators)")

In [8]:
minibatches = memory.get_minibatches(next_value, next_done)

obs = minibatches[0].observations.cpu() # shape [minibatch_size, obs_shape=4]
dones = minibatches[0].dones.cpu() # shape [minibatch_size,]

plot_cartpole_obs_and_dones(obs, dones, title="CartPole experiences for single minibatch (dotted lines = termination)")

In [9]:
class PPOAgent(nn.Module):
    critic: nn.Sequential
    actor: nn.Sequential

    def __init__(self, args: PPOArgs, envs: gym.vector.SyncVectorEnv):
        super().__init__()
        self.args = args
        self.envs = envs

        # Keep track of global number of steps taken by agent
        self.step = 0

        # Get actor and critic networks
        self.actor, self.critic = get_actor_and_critic(envs, mode=args.mode)

        # Define our first (obs, done), so we can start adding experiences to our replay memory
        self.next_obs = t.tensor(envs.reset()).to(device, dtype=t.float)
        self.next_done = t.zeros(envs.num_envs).to(device, dtype=t.float)

        # Create our replay memory
        self.memory = ReplayMemory(args, envs)


    def play_step(self) -> list[dict]:
        '''
        Carries out a single interaction step between the agent and the environment, and adds results to the replay memory.

        Returns the list of info dicts returned from `self.envs.step`.
        '''
        # Get newest observations (this is where we start from)
        obs = self.next_obs
        dones = self.next_done

        # carry out single itteration between the agent and the environment 
        with t.inference_mode():
            logits = self.actor(obs)

        probs = Categorical(logits=logits)
        actions = probs.sample()

        # take a step in the evnironemtn absed on the sampled action
        next_obs, rewards, next_dones, infos = self.envs.step(actions.cpu().numpy())

        # calculate logprobs and add them to the memory
        logprobs = probs.log_prob(actions)

        with t.inference_mode():
            values = self.critic(obs).flatten()

        # add the results to the reply memory
        self.memory.add(obs, actions, logprobs, values, rewards, dones)
        self.next_obs = t.from_numpy(next_obs).to(device, dtype=t.float)
        self.next_done = t.from_numpy(next_dones).to(device, dtype=t.float)

        self.step += self.envs.num_envs

        return infos 

    def get_minibatches(self) -> list[ReplayMinibatch]:
        '''
        Gets minibatches from the replay memory.
        '''
        with t.inference_mode():
            next_value = self.critic(self.next_obs).flatten()
        return self.memory.get_minibatches(next_value, self.next_done)


tests.test_ppo_agent(PPOAgent)

All tests in `test_agent` passed!


In [10]:
def calc_clipped_surrogate_objective(
    probs: Categorical,
    mb_action: Int[Tensor, "minibatch_size *action_shape"],
    mb_advantages: Float[Tensor, "minibatch_size"],
    mb_logprobs: Float[Tensor, "minibatch_size"],
    clip_coef: float,
    eps: float = 1e-8
) -> Float[Tensor, ""]:
    '''Return the clipped surrogate objective, suitable for maximisation with gradient ascent.

    probs:
        a distribution containing the actor's unnormalized logits of shape (minibatch_size, num_actions)
    mb_action:
        what actions actions were taken in the sampled minibatch
    mb_advantages:
        advantages calculated from the sampled minibatch
    mb_logprobs:
        logprobs of the actions taken in the sampled minibatch (according to the old policy)
    clip_coef:
        amount of clipping, denoted by epsilon in Eq 7.
    eps:
        used to add to std dev of mb_advantages when normalizing (to avoid dividing by zero)
    '''
    assert mb_action.shape == mb_advantages.shape == mb_logprobs.shape
    
    logits_diff = probs.log_prob(mb_action) - mb_logprobs
    r_theta = t.exp(logits_diff)
    
    mb_advantages = (mb_advantages - mb_advantages.mean()) / (mb_advantages.std() + eps)
    
    non_clipped = r_theta * mb_advantages
    clipped = t.clamp(r_theta, 1.0 - clip_coef, 1.0 + clip_coef) * mb_advantages
    
    clipped_surrogate_objective = t.min(non_clipped, clipped).mean()
    return clipped_surrogate_objective


tests.test_calc_clipped_surrogate_objective(calc_clipped_surrogate_objective)

All tests in `test_calc_clipped_surrogate_objective` passed.


In [11]:
def calc_value_function_loss(
    values: Float[Tensor, "minibatch_size"],
    mb_returns: Float[Tensor, "minibatch_size"],
    vf_coef: float
) -> Float[Tensor, ""]:
    '''Compute the value function portion of the loss function.

    values:
        the value function predictions for the sampled minibatch (using the updated critic network)
    mb_returns:
        the target for our updated critic network (computed as `advantages + values` from the old network)
    vf_coef:
        the coefficient for the value loss, which weights its contribution to the overall loss. Denoted by c_1 in the paper.
    '''
    assert values.shape == mb_returns.shape
    return vf_coef * (values - mb_returns).pow(2).mean()


tests.test_calc_value_function_loss(calc_value_function_loss)

All tests in `test_calc_value_function_loss` passed!


In [12]:
def calc_entropy_bonus(probs: Categorical, ent_coef: float):
    '''Return the entropy bonus term, suitable for gradient ascent.

    probs:
        the probability distribution for the current policy
    ent_coef:
        the coefficient for the entropy loss, which weights its contribution to the overall objective function. Denoted by c_2 in the paper.
    '''
    return ent_coef * probs.entropy().mean()

tests.test_calc_entropy_bonus(calc_entropy_bonus)

All tests in `test_calc_entropy_bonus` passed!


In [13]:
class PPOScheduler:
    def __init__(self, optimizer: Optimizer, initial_lr: float, end_lr: float, total_training_steps: int):
        self.optimizer = optimizer
        self.initial_lr = initial_lr
        self.end_lr = end_lr
        self.total_training_steps = total_training_steps
        self.n_step_calls = 0

    def step(self):
        '''Implement linear learning rate decay so that after total_training_steps calls to step, the learning rate is end_lr.

        Do this by directly editing the learning rates inside each param group (i.e. `param_group["lr"] = ...`), for each param
        group in `self.optimizer.param_groups`.
        '''
        self.n_step_calls += 1
        frac = self.n_step_calls / self.total_training_steps
        assert frac <= 1
        for param_group in self.optimizer.param_groups:
            param_group["lr"] = self.initial_lr + frac * (self.end_lr - self.initial_lr)


def make_optimizer(agent: PPOAgent, total_training_steps: int, initial_lr: float, end_lr: float) -> tuple[optim.Adam, PPOScheduler]:
    '''Return an appropriately configured Adam with its attached scheduler.'''
    optimizer = optim.Adam(agent.parameters(), lr=initial_lr, eps=1e-5, maximize=True)
    scheduler = PPOScheduler(optimizer, initial_lr, end_lr, total_training_steps)
    return (optimizer, scheduler)


tests.test_ppo_scheduler(PPOScheduler)

All tests in `test_ppo_scheduler` passed!


#### Training Loop

```Python
initialise PPOArgs and PPOTrainer objects
for phase in args.total_phases:
    trainer.rollout_phase()
    trainer.learning_phase()
    # log any relevant variables
```

In [14]:
class PPOTrainer:

    def __init__(self, args: PPOArgs):
        set_global_seeds(args.seed)
        self.args = args
        self.run_name = f"{args.env_id}__{args.exp_name}__{args.seed}__{int(time.time())}"
        self.envs = gym.vector.SyncVectorEnv([make_env(args.env_id, args.seed + i, i, args.capture_video, self.run_name, args.mode) for i in range(args.num_envs)])
        self.agent = PPOAgent(self.args, self.envs).to(device)
        self.optimizer, self.scheduler = make_optimizer(self.agent, self.args.total_training_steps, self.args.learning_rate, 0.0)


    def rollout_phase(self) -> int | None:
        '''
        This function populates the memory with a new set of experiences, using `self.agent.play_step`
        to step through the environment. It also returns the episode length of the most recently terminated
        episode (used in the progress bar readout).
        '''
        last_episode_len = None
        for _ in range(self.args.num_steps):
            infos = self.agent.play_step()
            for info in infos:
                if "episode" in info.keys():
                    last_episode_len = info["episode"]["l"]
                    last_episode_return= info["episode"]["r"]
                    if self.args.use_wandb:
                        wandb.log(
                            {
                                "episode_length": last_episode_len,
                                "episode_return": last_episode_return,
                            },
                            step=self.agent.step,
                        )
        return last_episode_len


    def learning_phase(self) -> None:
        '''
        This function does the following:

            - Generates minibatches from memory
            - Calculates the objective function, and takes an optimization step based on it
            - Clips the gradients (see detail #11)
            - Steps the learning rate scheduler
        '''
        minibatches = self.agent.get_minibatches()
        for minib in minibatches:
            objective_fn = self.compute_ppo_objective(minib)
            objective_fn.backward()
            nn.utils.clip_grad_norm_(self.agent.parameters(), self.args.max_grad_norm)
            self.optimizer.step()
            self.optimizer.zero_grad()
            self.optimizer.step()


    def compute_ppo_objective(self, minibatch: ReplayMinibatch) -> Float[Tensor, ""]:
        '''
        Handles learning phase for a single minibatch. Returns objective function to be maximized.
        '''
        logits = self.agent.actor(minibatch.observations)
        probs = Categorical(logits=logits)
        values = self.agent.critic(minibatch.observations).squeeze()
        clipped_surrogate_objective = calc_clipped_surrogate_objective(
            probs, minibatch.actions, minibatch.advantages, minibatch.logprobs, self.args.clip_coef)
        value_loss = calc_value_function_loss(values, minibatch.returns, self.args.vf_coef)
        entropy_bonus = calc_entropy_bonus(probs, self.args.ent_coef)
        
        total_objective_function = clipped_surrogate_objective - value_loss + entropy_bonus

        with t.inference_mode():
            newlogprob = probs.log_prob(minibatch.actions)
            logratio = newlogprob - minibatch.logprobs
            ratio = logratio.exp()
            approx_kl = (ratio - 1 - logratio).mean().item()
            clipfracs = [((ratio - 1.0).abs() > self.args.clip_coef).float().mean().item()]
            if self.args.use_wandb:
                wandb.log(
                    dict(
                        total_steps=self.agent.step,
                        valeus=values.mean().itme(),
                        learning_rate=self.scheduler.optimizer.param_groups[0]["lr"],
                        value_loss=value_loss.item(),
                        clipped_surrogate_objective=clipped_surrogate_objective.item(),
                        approx_kl=approx_kl,
                        clipfrac=np.mean(clipfracs),
                    ),
                    step=self.agent.step,
                )
        return total_objective_function

    def train(self) -> None:

        if args.use_wandb: wandb.init(
            project=self.args.wandb_project_name,
            entity=self.args.wandb_entity,
            name=self.run_name,
            monitor_gym=self.args.capture_video
        )

        progress_bar = tqdm(range(self.args.total_phases))

        for epoch in progress_bar:

            last_episode_len = self.rollout_phase()
            if last_episode_len is not None:
                progress_bar.set_description(f"Epoch {epoch:02}, Episode length: {last_episode_len}")

            self.learning_phase()

        self.envs.close()
        if self.args.use_wandb:
            wandb.finish()

In [16]:
# def test_probe(probe_idx: int):
    '''
    Tests a probe environment by training a network on it & verifying that the value functions are
    in the expected range.
    '''
    # Train our network
    args = PPOArgs(
        env_id=f"Probe{probe_idx}-v0",
        exp_name=f"test-probe-{probe_idx}",
        total_timesteps=[5000, 5000, 10000, 20000, 20000][probe_idx-1],
        learning_rate=0.001,
        capture_video=False,
        use_wandb=False,
    )
    trainer = PPOTrainer(args)
    trainer.train()
    agent = trainer.agent

    # Get the correct set of observations, and corresponding values we expect
    obs_for_probes = [[[0.0]], [[-1.0], [+1.0]], [[0.0], [1.0]], [[0.0]], [[0.0], [1.0]]]
    expected_value_for_probes = [[[1.0]], [[-1.0], [+1.0]], [[args.gamma], [1.0]], [[1.0]], [[1.0], [1.0]]]
    expected_probs_for_probes = [None, None, None, [[0.0, 1.0]], [[1.0, 0.0], [0.0, 1.0]]]
    tolerances = [1e-3, 1e-3, 1e-3, 2e-3, 2e-3]
    obs = t.tensor(obs_for_probes[probe_idx-1]).to(device)

    # Calculate the actual value & probs, and verify them
    with t.inference_mode():
        value = agent.critic(obs)
        probs = agent.actor(obs).softmax(-1)
    expected_value = t.tensor(expected_value_for_probes[probe_idx-1]).to(device)
    t.testing.assert_close(value, expected_value, atol=tolerances[probe_idx-1], rtol=0)
    expected_probs = expected_probs_for_probes[probe_idx-1]
    if expected_probs is not None:
        t.testing.assert_close(probs, t.tensor(expected_probs).to(device), atol=tolerances[probe_idx-1], rtol=0)
    print("Probe tests passed!\n")


for probe_idx in range(1, 6):
    test_probe(probe_idx)

IndentationError: unexpected indent (3568294602.py, line 2)

In [17]:
args = PPOArgs(use_wandb=True)
trainer = PPOTrainer(args)
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 84rt. Use `wandb login --relogin` to force relogin
Thread HandlerThread:
Traceback (most recent call last):
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 49, in run
    self._run()
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/internal/internal_util.py", line 100, in _run
    self._process(record)
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/internal/internal.py", line 280, in _process
    self._hm.handle(record)
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/internal/handler.py", line 136, in handle
    handler(record)
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/internal/handler.py", line 146, in handle_request
    handler(record)
  File "/home/bart/.local/lib/python3.10/site-packages/wan

Problem at: /tmp/ipykernel_25185/3709565458.py 92 train


Traceback (most recent call last):
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 1133, in init
    run = wi.init()
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 787, in init
    run_start_result = run_start_handle.wait(timeout=30)
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/lib/mailbox.py", line 271, in wait
    raise MailboxError("transport failed")
wandb.errors.MailboxError: transport failed
wandb: ERROR Abnormal program exit


: 

In [ ]:
from gym.envs.classic_control.cartpole import CartPoleEnv

class EasyCart(CartPoleEnv):
    def step(self, action):
        (obs, reward, done, info) = super().step(action)
        x, v, theta, omega = obs
        
        raw_1 = abs(theta / 0.2095)

        raw_2 = 1 - abs(x / 2.4)

        raw_new = (raw_1 + raw_2) / 2

        return (obs, raw_new, done, info)



gym.envs.registration.register(id="EasyCart-v0", entry_point=EasyCart, max_episode_steps=500)
args = PPOArgs(env_id="EasyCart-v0", use_wandb=True)
trainer = PPOTrainer(args)
trainer.train()

Problem at: /tmp/ipykernel_24100/3709565458.py 92 train


Traceback (most recent call last):
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 1133, in init
    run = wi.init()
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/wandb_init.py", line 593, in init
    manager._inform_init(settings=self.settings, run_id=self.settings.run_id)
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/wandb_manager.py", line 174, in _inform_init
    svc_iface._svc_inform_init(settings=settings, run_id=run_id)
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/service/service_sock.py", line 38, in _svc_inform_init
    self._sock_client.send(inform_init=inform_init)
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 211, in send
    self.send_server_request(server_req)
  File "/home/bart/.local/lib/python3.10/site-packages/wandb/sdk/lib/sock_client.py", line 155, in send_server_request
    self._send_message(msg)
  File "/home/bart/.local/lib

Exception: problem

In [ ]:
%pip install autorom[accept-rom-license]
%pip install ale-py

Defaulting to user installation because normal site-packages is not writeable
  Using cached AutoROM.accept-rom-license-0.6.1.tar.gz (434 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.accept-rom-license: filename=AutoROM.accept_rom_license-0.6.1-py3-none-any.whl size=440873 sha256=24ca58a1920bc9400d2fb97311b18ec58f18e6ef538c7c0748e7ad1622c04e4d
  Stored in directory: /home/bart/.cache/pip/wheels/6b/1b/ef/a43ff1a2f1736d5711faa1ba4c1f61be1131b8899e6a057811
Successfully built AutoROM.accept-rom-license
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.2 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
env = gym.make("ALE/Breakout-v5")
print(env.action_space)

Exception ignored in: <function VectorEnv.__del__ at 0x7037f02cad40>
Traceback (most recent call last):
  File "/home/bart/.local/lib/python3.10/site-packages/gym/vector/vector_env.py", line 215, in __del__
    self.close()
  File "/home/bart/.local/lib/python3.10/site-packages/gym/vector/vector_env.py", line 193, in close
    self.close_extras(**kwargs)
  File "/home/bart/.local/lib/python3.10/site-packages/gym/vector/sync_vector_env.py", line 181, in close_extras
    [env.close() for env in self.envs]
  File "/home/bart/.local/lib/python3.10/site-packages/gym/vector/sync_vector_env.py", line 181, in <listcomp>
    [env.close() for env in self.envs]
  File "/home/bart/.local/lib/python3.10/site-packages/gym/wrappers/record_video.py", line 121, in close
    self.close_video_recorder()
  File "/home/bart/.local/lib/python3.10/site-packages/gym/wrappers/record_video.py", line 116, in close_video_recorder
    self.video_recorder.close()
  File "/home/bart/.local/lib/python3.10/site-packag

NamespaceNotFound: Namespace `ALE` does not exist. Have you installed the proper package for `ALE`?